# Table of Contents
1. Creating UCCSD ansatz circuit for LiH
2. Optimizing circuit
3. Mapping circuit to nearest neighbor graph
-----
## 1. Creating UCCSD Ansatz Circuit for LiH

In [1]:
import sys
sys.path.append('..')
from fqc import uccsd, util
import numpy as np

In [4]:
theta_vector = [np.random.rand() * 2 * np.pi for _ in range(8)]
print(theta_vector)

[5.768207251099332, 4.6910919798580695, 1.5257947739159419, 2.5844508299301863, 2.5027078490614114, 2.4629881311449258, 2.765490721732298, 4.712423723784896]


In [5]:
circuit = uccsd.get_uccsd_circuit('LiH', theta_vector)
circuit.draw(output='text', line_length=4000)

In [6]:
util.circuitutil.get_max_pulse_time(circuit)

1501.699999999997

In [7]:
unoptimized_unitary = util.get_unitary(circuit)

## 2. Optimizing circuit

In [8]:
util.circuitutil.merge_rotation_gates(circuit)

In [9]:
circuit.draw(output='text', line_length=4000)

In [10]:
util.circuitutil.get_max_pulse_time(circuit)

1392.7999999999972

In [11]:
optimized_unitary = util.get_unitary(circuit)
print(np.array_equal(optimized_unitary * -1, unoptimized_unitary))  # (equal up to -1 global phase)

True


## 3. Mapping circuit to nearest neighbor graph

In [12]:
coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(2, 2)

In [13]:
mapped_circuit = util.circuitutil.impose_swap_coupling(circuit, coupling_list)

In [14]:
mapped_circuit.draw(output='text', line_length=4000)

In [15]:
util.circuitutil.get_max_pulse_time(mapped_circuit)

1508.4999999999948

In [16]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [20]:
import numpy as np
import os, sys, math
from datetime import datetime
sys.path.append('..'); import config

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [17]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 1000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}
reg_coeffs = {}

In [18]:
N = 4
connected_qubit_pairs = util.circuitutil.get_nearest_neighbor_coupling_list(2, 2, directed=False)
H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d)
U = transmon_gate(optimized_unitary, d)

In [21]:
total_time = 50.3
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

data saved at: /project/ftchong/qoc/pranav/00000_Feb18.h5
Using 6 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.96e-01; Runtime: 16.2s; Iterations = 0, grads =   9.499e-03, unitary_metric = 0.99928
Error = :9.36e-01; Runtime: 96.0s; Iterations = 100, grads =   2.711e-02, unitary_metric = 1.00018
Error = :1.96e-01; Runtime: 174.0s; Iterations = 200, grads =   8.454e-02, unitary_metric = 0.99981
Error = :8.66e-02; Runtime: 252.5s; Iterations = 300, grads =   9.031e-02, unitary_metric = 0.99974
Error = :2.34e-02; Runtime: 331.3s; Iterations = 400, grads =   6.808e-03, unitary_metric = 0.99971
Error = :7.23e-03; Runtime: 410.6s; Iterations = 500, grads =   9.047e-04, unitary_metric = 0.99967
Error = :3.77e-03;

In [ ]:
# allow 2000 iterations-->total_time is much smaller :(

d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 2000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}
reg_coeffs = {}
total_time = 44.4
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

data saved at: /project/ftchong/qoc/pranav/00012_Feb17.h5
Using 6 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.88e-01; Runtime: 1.5s; Iterations = 0, grads =   6.667e-03, unitary_metric = 0.99936
Error = :4.00e-01; Runtime: 76.4s; Iterations = 100, grads =   1.120e+00, unitary_metric = 0.99951
Error = :1.48e-01; Runtime: 151.7s; Iterations = 200, grads =   1.797e+00, unitary_metric = 0.99937
Error = :9.45e-02; Runtime: 224.7s; Iterations = 300, grads =   6.420e-01, unitary_metric = 0.99935
Error = :3.80e-02; Runtime: 297.6s; Iterations = 400, grads =   2.477e-01, unitary_metric = 0.99932
Error = :4.62e-02; Runtime: 369.7s; Iterations = 500, grads =   1.442e-02, unitary_metric = 0.99927
Error = :3.52e-02; 